In [ ]:
import os
import pandas as pd
import numpy as np
import regex as re
import mytrain_lib as ml
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

In [ ]:
def plot_results(error,accuracy_train,accuracy_test,confusion_matrix, hyperparams):
    confusion_matrix = np.array(confusion_matrix)
    accuracy_test    = np.array(accuracy_test)
    accuracy_train   = np.array(accuracy_train)
    error            = np.array(error)

    acc_test_lastepoch = accuracy_test[:,:,-1]  # only interested in last epoch

    # best models of each configuration
    best_cv          = acc_test_lastepoch.argmax(axis=1)
    best_config_cv   = np.unique(acc_test_lastepoch.argmax(axis=0))

    # best configurations are:
    print('config','\t', 'accuracy_test\t', '\taccuracy_train\t', '\terror')

    for c in best_config_cv:
        print(c,'\t', accuracy_test[c,best_cv[c],-1]
                    , accuracy_train[c,best_cv[c],-1]
                    , error[c,best_cv[c],-1])

    temp = datetime.now().strftime("_%m%d_%H%M%S")

    for i,c in enumerate(best_config_cv):
        print(f'Config of {c} - Fold {best_cv[c]}: {hyperparams[c]}')
        ml.dispConfusionMatrix(confusion_matrix[c,best_cv[c]],
                        f'Confusion Matrix: MLP 1x5 SGD {hyperparams[c]}',
                        f'confmat_mlp5_SGD_t{temp}_id{i}', save=True)

        

In [ ]:
path_train      = 'F://TFG//datasets//data_train//'
path_graphs     = 'F://TFG//graphs//plot_results//'
path_results    = 'F://TFG//results//'

In [ ]:
data        = pd.read_csv(path_train+'training_features_DF.csv',sep=';',index_col='wyId')
raw_Data    = pd.read_json('F://TFG//datasets/raw_datasets//RAW_partidos.json').set_index('wyId')

In [ ]:
ep, pca, opt = [10,20], [3,5,10], ['sgd','adam']
params = (np.array(np.meshgrid(ep,pca,opt)).T.reshape((-1,3)))

acc_res, acc_err = {}, {}
r_acc = re.compile(r'(acctest__)(\w)*(.npy)')
r_err = re.compile(r'(error__)(\w)*(.npy)')

for p in params:
    path_nopca  = f'mlp15_{p[2]}_ep{p[0]}'
    path_pca    = f'mlp15_{p[2]}_ep{p[0]}_pca{p[1]}'

    files_nopca = os.listdir(path_results+path_nopca)
    files_pca   = os.listdir(path_results+path_pca)

    if p[1]=='3': 
        file_name = list(filter(r_acc.match, files_nopca))[0]
        acc_res[path_nopca] = np.load(path_results+path_nopca+'//'+file_name)
        file_name = list(filter(r_err.match, files_nopca))[0]
        acc_err[path_nopca] = np.load(path_results+path_nopca+'//'+file_name)
        
    file_name = list(filter(r_acc.match, files_pca))[0]
    acc_res[path_pca] = np.load(path_results+path_pca+'//'+file_name)
    file_name = list(filter(r_err.match, files_pca))[0]
    acc_err[path_pca] = np.load(path_results+path_pca+'//'+file_name)

In [ ]:
xticks = []

for p in params:
    if p[1]=='3':
        xticks.append(f'ep={p[0]}')
    xticks.append('ep={}\npca={}'.format(p[0],p[1]))

## Accuracy on Experiments

In [ ]:
max_values = [np.max(acc_res[k][:,:,-1]) for k in acc_res.keys()]
mean_values = [np.mean(acc_res[k][:,:,-1]) for k in acc_res.keys()]

color = ['#1183C4','#17CA45','#0E5F8E','#108C30']

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(12,10))

fig.suptitle('Accuracy on experiments',fontsize=18)

ax1.set_title('Max Accuracy')
ax1.bar(x=list(acc_res.keys()),height=max_values,color=[color[i>7] for i in range(16)])
ax1.set_xlabel('experiments'); ax1.set_ylabel('accuracy')
ax1.set_ylim([0,1])
ax1.set_xticklabels('')

ax2.set_title('Mean Accuracy')
ax2.bar(x=list(acc_res.keys()),height=mean_values,color=[color[(i>7)+2] for i in range(16)])
ax2.set_xlabel('experiments'); ax2.set_ylabel('accuracy')
ax2.set_xticklabels(ax2.get_xticks(), rotation = 0)
ax2.set_ylim([0,1])
ax2.set_xticklabels(xticks[:len(mean_values)],fontsize=8)

plt.savefig(path_graphs + 'accuracy_expPCA.jpg', format='jpg', dpi=200)
plt.show()

## Error on experiments

In [ ]:
max_values = [np.min(acc_err[k][:,:,-1]) for k in acc_res.keys()]
mean_values = [np.mean(acc_err[k][:,:,-1]) for k in acc_res.keys()]

color = ['#D14444','#B5189D','#6F2424','#630E56']

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(12,10))

fig.suptitle('Error on experiments',fontsize=18)

ax1.set_title('Min error')
ax1.bar(x=list(acc_err.keys()),height=max_values,color=[color[i>7] for i in range(16)])
ax1.set_xlabel('experiments'); ax1.set_ylabel('error')
ax1.set_ylim([0,1.2])
ax1.set_xticklabels('')

ax2.set_title('Mean error')
ax2.bar(x=list(acc_err.keys()),height=mean_values,color=[color[(i>7)+2] for i in range(16)])
ax2.set_xlabel('experiments'); ax2.set_ylabel('error')
ax2.set_xticklabels(ax2.get_xticks(), rotation = 0)
ax2.set_ylim([0,1.2])
ax2.set_xticklabels(xticks,fontsize=8)

plt.savefig(path_graphs + 'error_expPCA.jpg', format='jpg', dpi=200)
plt.show()